In [7]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# playaround gpd: read a kml file and convert to gee polygon

In [81]:
import geopandas as gpd
import fiona 

# Enable fiona driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

polygon = gpd.read_file(r'.\files\munich.kml', driver='KML')
# print(polygon)
geom = polygon.iloc[0:1, :]
# print(geom)

'''convert to gee feature'''
jsonDict = eval(geom.to_json())
geojsonDict = jsonDict['features'][0]

cor=geojsonDict['geometry']['coordinates']
print()
         
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)
print(aoi)
            


ee.Geometry({
  "type": "Invocation",
  "arguments": {
    "coordinates": [
      [
        11.3602677630673,
        48.02751971264227
      ],
      [
        11.80137465433197,
        48.03234154343271
      ],
      [
        11.79876828408382,
        48.23050987210912
      ],
      [
        11.35590852476603,
        48.23019011206229
      ],
      [
        11.3602677630673,
        48.02751971264227
      ]
    ],
    "geodesic": false
  },
  "functionName": "GeometryConstructors.Polygon"
})


# playaround gee

In [ ]:
Map = geemap.Map()
Map

In [79]:
# Display the polygons by adding them to the map.
Map.centerObject(aoi,10)
Map.addLayer(aoi, {'color': 'FF0000'})

In [62]:
Map.add_landsat_ts_gif(label='', start_year=1985, bands=['Red', 'Green','Blue'], frames_per_second=5)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\User\Downloads\landsat_ts_jag.gif
Adding animated text to GIF ...
Adding GIF to the map ...


In [80]:
#To create a Landsat timelapse animation and add it to the Map:  
Map.add_landsat_ts_gif(label='roi', roi=aoi, start_year=1985, bands=['Red', 'Green','Blue'], frames_per_second=1)
#

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\User\Downloads\landsat_ts_sbu.gif
Adding animated text to GIF ...
Adding GIF to the map ...


In [76]:
imgC=geemap.landsat_timeseries(roi=aoi, start_year=1984, end_year=2019, start_date='06-10', end_date='09-20')
#print(imgC.getInfo())
count = imgC.size().getInfo()
print("Count: ", count)
vis = {'bands': ['Red', 'Green','Blue']}
Map.addLayer(imgC,vis)

Count:  36


In [ ]:
geemap.ee_export_image_collection(ee_object, output, scale, crs, region, file_per_band)

# 

In [ ]:
import os

# aoi = ee.Geometry.Polygon(
#   [[[-110.8, 44.7],
#     [-110.8, 44.6],
#     [-110.6, 44.6],
#     [-110.6, 44.7]]], None, False)

loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

print(collection.aggregate_array('system:index').getInfo())

geemap.ee_export_image_collection(collection, out_dir=out_dir)

#https://github.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb